In [3]:
import pandas as pd
import xgboost as xgb
import numpy as np

In [4]:
spammer_order = "../feature.json"
raw_file = pd.read_json(open(spammer_order), orient='records')
raw_file.head()

,content_length,followers_num,following_num,hashtag,id,is_spammer,mention,post_num,reputation,text_similarity,time_interval_mean,time_interval_var,url
0,60.480000,179,358,15,1214054254,no,7,1005,0.500000,0.034592,30972.857143,8.239935e+08,1
1,48.510204,312,180,10,2158838150,no,3,1819,1.733333,0.026582,33023.000000,7.491242e+08,0
2,59.979592,704,715,18,1465038162,no,3,3542,0.984615,0.027306,36454.020833,6.229491e+08,2
3,102.660000,5,387,27,5203407567,no,19,340,0.012920,0.031989,11511.346939,7.102748e+08,4
4,54.918367,93,455,7,3944654540,yes,3,371,0.204396,0.055030,26378.770833,7.995144e+08,38


In [5]:
raw_file.describe()

,id,content_length,hashtag,late_night_times,figure_at,reputation,followee_num,followers_num,figure_at_every,following_num,...,time_interval_mean,test_similarity,figure_RRT,follow_ratio,day_interval_variance,text_similarity,time_interval_var,average_comm,post_num,mention
count,1.566000e+03,1566.000000,1566.000000,1566.0,1566.0,1566.000000,1566.0,1566.000000,1566.0,1566.000000,...,1566.000000,1566.000000,1566.0,1566.0,1566.0,1566.0,1.566000e+03,1566.0,1566.000000,1566.000000
mean,2.967337e+09,65.434807,10.814815,0.0,0.0,2.009261,0.0,558.009579,0.0,683.523627,...,29051.843270,0.062515,0.0,0.0,0.0,0.0,7.228952e+08,0.0,1460.061941,6.033206
std,1.184598e+09,22.656497,9.773090,0.0,0.0,28.911290,0.0,1701.318891,0.0,626.339048,...,10882.857669,0.091577,0.0,0.0,0.0,0.0,2.586880e+08,0.0,3472.917546,7.110370
min,1.005384e+09,5.000000,0.000000,0.0,0.0,0.009146,0.0,3.000000,0.0,1.000000,...,6.204082,0.020051,0.0,0.0,0.0,0.0,0.000000e+00,0.0,2.000000,0.000000
25%,2.021067e+09,49.580833,3.000000,0.0,0.0,0.317523,0.0,133.250000,0.0,220.000000,...,22165.185566,0.030094,0.0,0.0,0.0,0.0,6.211694e+08,0.0,294.750000,1.000000
50%,2.727800e+09,62.160000,8.000000,0.0,0.0,0.579435,0.0,246.000000,0.0,412.000000,...,30547.811437,0.037440,0.0,0.0,0.0,0.0,7.618443e+08,0.0,669.000000,4.000000
75%,3.540640e+09,78.075306,16.000000,0.0,0.0,1.051232,0.0,496.500000,0.0,970.000000,...,37380.515306,0.055187,0.0,0.0,0.0,0.0,8.798472e+08,0.0,1520.750000,8.000000
max,5.397886e+09,181.250000,49.000000,0.0,0.0,1129.000000,0.0,33924.000000,0.0,3229.000000,...,65396.666667,1.000000,0.0,0.0,0.0,0.0,1.731597e+09,0.0,102353.000000,50.000000


In [7]:
data = raw_file.drop(['is_spammer'], axis=1)
target = raw_file[['is_spammer']]


In [8]:
data

,content_length,followers_num,following_num,hashtag,id,mention,post_num,reputation,text_similarity,time_interval_mean,time_interval_var,url
0,60.480000,179,358,15,1214054254,7,1005,0.500000,0.034592,30972.857143,8.239935e+08,1
1,48.510204,312,180,10,2158838150,3,1819,1.733333,0.026582,33023.000000,7.491242e+08,0
2,59.979592,704,715,18,1465038162,3,3542,0.984615,0.027306,36454.020833,6.229491e+08,2
3,102.660000,5,387,27,5203407567,19,340,0.012920,0.031989,11511.346939,7.102748e+08,4
4,54.918367,93,455,7,3944654540,3,371,0.204396,0.055030,26378.770833,7.995144e+08,38
5,50.800000,253,1850,0,5055635073,1,467,0.136757,0.030800,37345.510204,1.169251e+08,0
6,73.931818,59,120,6,5037076803,2,620,0.491667,0.051178,18005.395349,1.072603e+09,2
7,59.680000,116,384,4,2703197560,42,401,0.302083,0.415591,14511.469388,6.036607e+08,4
8,61.840000,257,516,4,2172215857,1,631,0.498062,0.029096,20287.081633,8.519779e+08,3
9,69.666667,6323,1810,12,1223815707,4,1479,3.493370,0.074021,33078.620000,1.000798e+09,18


In [9]:
target

,is_spammer
0,no
1,no
2,no
3,no
4,yes
5,yes
6,no
7,yes
8,no
9,yes


In [8]:
from sklearn import preprocessing

def norm(data, column_name):
    x = data[column_name].values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    data_norm = pd.DataFrame(x_scaled)
    data[column_name] = data_norm
    return data

In [9]:
norm(data, 'time_interval_mean')
norm(data, 'time_interval_var')

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

,hashtag,reputation,post_num,test_similarity,url,time_interval_mean,time_interval_var,followers_num,following_num,content_length
0,15,0.500000,1005,0.034592,1,0.473565,0.475858,179,358,60.480000
1,10,1.733333,1819,0.026582,0,0.504918,0.432620,312,180,48.510204
2,18,0.984615,3542,0.027306,2,0.557383,0.359739,704,715,59.979592
3,27,0.012920,340,0.031989,4,0.175945,0.410185,5,387,102.660000
4,7,0.204396,371,0.055030,38,0.403309,0.461721,93,455,54.918367
5,0,0.136757,467,0.030800,0,0.571021,0.067524,253,1850,50.800000
6,6,0.491667,620,0.051178,2,0.275257,0.619430,59,120,73.931818
7,4,0.302083,401,0.415591,4,0.221825,0.348615,116,384,59.680000
8,4,0.498062,631,0.029096,3,0.310150,0.492019,257,516,61.840000
9,12,3.493370,1479,0.074021,18,0.505768,0.577963,6323,1810,69.666667


In [10]:
target

,is_spammer
0,no
1,no
2,no
3,no
4,yes
5,yes
6,no
7,yes
8,no
9,yes


In [11]:
from sklearn.model_selection import GridSearchCV
target['is_spammer']=target['is_spammer'].map(dict(yes=1, no=0))

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


# 2. Machine Learning Model

## 2.1 Classical Models

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

In [13]:
svc = SVC(kernel='rbf', gamma=1.0)
knc = KNeighborsClassifier(n_neighbors=49)
mnb = MultinomialNB(alpha=0.2)
dtc = DecisionTreeClassifier(min_samples_split=7, random_state=111)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=31, random_state=111)
abc = AdaBoostClassifier(n_estimators=62, random_state=111)
bc = BaggingClassifier(n_estimators=9, random_state=111)
etc = ExtraTreesClassifier(n_estimators=9, random_state=111)

In [14]:
clfs = {'SVC' : svc,'KNeighborsClassifier' : knc, 'MultinomialNaiveBayes': mnb, 'DecisionTreeClassifier': dtc, 
        'LogisticRegression': lrc, 'RandomForestClassifier': rfc, 'AdaBoost': abc, 'BaggingClassifier': bc, 
        'ExtraTreesClassifier': etc}

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=5)

# from sklearn.model_selection import ShuffleSplit
# cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
from sklearn.utils import column_or_1d
target = column_or_1d(target, warn=True)
pred_scores = []
for k,v in clfs.items():
    results = cross_val_score(v, data, target, cv=cv)
    print(k, results, results.mean())
    pred_scores.append((k, results))

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


LogisticRegression [ 0.73248408  0.74440895  0.73162939  0.73482428  0.69648562] 0.727966463849
ExtraTreesClassifier [ 0.69426752  0.73801917  0.73162939  0.69329073  0.66773163] 0.704987688488
MultinomialNaiveBayes [ 0.68789809  0.59744409  0.61661342  0.67092652  0.66453674] 0.647483771189
BaggingClassifier [ 0.73248408  0.69009585  0.72523962  0.69009585  0.68690096] 0.704963268961
SVC [ 0.47133758  0.52715655  0.50479233  0.52715655  0.50159744] 0.506408091003
KNeighborsClassifier [ 0.68471338  0.64536741  0.67092652  0.7284345   0.65495208] 0.676878777396
DecisionTreeClassifier [ 0.70063694  0.63897764  0.65814696  0.65175719  0.66134185] 0.66217211697
AdaBoost [ 0.72611465  0.73801917  0.7028754   0.71565495  0.70926518] 0.718385869233
RandomForestClassifier [ 0.78025478  0.73162939  0.73801917  0.77955272  0.69968051] 0.745827313241


In [16]:
df = pd.DataFrame.from_items(pred_scores,orient='index', columns=['Score'])
df

,Score
LogisticRegression,0.732484
ExtraTreesClassifier,0.694268
MultinomialNaiveBayes,0.687898
BaggingClassifier,0.732484
SVC,0.471338
KNeighborsClassifier,0.684713
DecisionTreeClassifier,0.700637
AdaBoost,0.726115
RandomForestClassifier,0.780255


## 2.2 XGBoost

In [17]:
# split to train and test sets
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(
    data, target, test_size=0.33, random_state=0)

train_X = train_X.as_matrix()
test_X = test_X.as_matrix()
train_y = train_y.as_matrix()

AttributeError: 'numpy.ndarray' object has no attribute 'as_matrix'

In [ ]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)
gbm.fit(train_X, train_y)
predictions = gbm.predict(test_X)

In [18]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

NameError: name 'predictions' is not defined

In [19]:
# or use 10 fold cross validation
from sklearn.model_selection import cross_val_score
model = xgb.XGBClassifier()

from sklearn.model_selection import KFold
cv = KFold(n_splits=5)

# from sklearn.model_selection import ShuffleSplit
# cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)

results = cross_val_score(model, data, target, cv=cv)
print(results)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
# initial mean accuracy 48.54%, deviation 31.44%

[ 0.76751592  0.7571885   0.74121406  0.76038339  0.69648562]
Accuracy: 74.46% (2.55%)


# 3. Parameter Tuning 

## 3.1 xgboost model parameters

In [20]:
def cross_val(alg):
#     kfold = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
    kfold = KFold(n_splits=5, random_state=0)
    scoring = ['accuracy', 'recall']
    results = cross_val_score(alg, data, target, cv=kfold, scoring='accuracy')
    print(results)
    print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    # initial mean accuracy 48.54%, deviation 31.44%

### 3.1.1. Find the number of estimators for a high learning rate

In [21]:
from xgboost.sklearn import XGBClassifier
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
cross_val(xgb1)

[ 0.71974522  0.73801917  0.71565495  0.7284345   0.70607029]
Accuracy: 72.16% (1.09%)


In [22]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}


from sklearn.model_selection import ShuffleSplit
shuffle = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)

gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=True, cv=shuffle)
gsearch1.fit(data,target)

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'max_depth': range(3, 10, 2), 'min_child_weight': range(1, 6, 2)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [23]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.81770, std: 0.01053, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.81637, std: 0.00999, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.82062, std: 0.01074, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.80746, std: 0.01182, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.80995, std: 0.01613, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.81041, std: 0.01488, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.80825, std: 0.00793, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.80722, std: 0.01043, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.80772, std: 0.01105, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.80787, std: 0.01202, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.80772, std: 0.01363, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.80515, std: 0.01134, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 3, 'min_child_weight': 5

In [24]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test2 = {
    'max_depth':[2,3,4],
    'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=3,
                                        min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch2.fit(data,target)

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=5, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'max_depth': [2, 3, 4], 'min_child_weight': [4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [25]:
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.82439, std: 0.00948, params: {'max_depth': 2, 'min_child_weight': 4},
  mean: 0.82640, std: 0.00966, params: {'max_depth': 2, 'min_child_weight': 5},
  mean: 0.82762, std: 0.00987, params: {'max_depth': 2, 'min_child_weight': 6},
  mean: 0.82042, std: 0.00989, params: {'max_depth': 3, 'min_child_weight': 4},
  mean: 0.82062, std: 0.01074, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.82130, std: 0.01030, params: {'max_depth': 3, 'min_child_weight': 6},
  mean: 0.81435, std: 0.01489, params: {'max_depth': 4, 'min_child_weight': 4},
  mean: 0.81530, std: 0.01094, params: {'max_depth': 4, 'min_child_weight': 5},
  mean: 0.81805, std: 0.01123, params: {'max_depth': 4, 'min_child_weight': 6}],
 {'max_depth': 2, 'min_child_weight': 6},
 0.82761544081026572)

In [26]:
param_test2b = {
    'min_child_weight':[6,8,10,12, 14, 16]
}
gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=2,
                                        min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch2b.fit(data,target)

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=6, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'min_child_weight': [6, 8, 10, 12, 14, 16]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [27]:
gsearch2b.grid_scores_, gsearch2b.best_params_, gsearch2b.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.82762, std: 0.00987, params: {'min_child_weight': 6},
  mean: 0.82614, std: 0.00774, params: {'min_child_weight': 8},
  mean: 0.82515, std: 0.00760, params: {'min_child_weight': 10},
  mean: 0.82182, std: 0.01094, params: {'min_child_weight': 12},
  mean: 0.82192, std: 0.00968, params: {'min_child_weight': 14},
  mean: 0.82150, std: 0.01148, params: {'min_child_weight': 16}],
 {'min_child_weight': 6},
 0.82761544081026572)

In [28]:
param_test2c = {
    'min_child_weight':[16, 20, 24, 28, 32, 36, 40]
}
gsearch2c = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=2,
                                        min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2c, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch2c.fit(data,target)

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=6, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'min_child_weight': [16, 20, 24, 28, 32, 36, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [29]:
gsearch2c.grid_scores_, gsearch2c.best_params_, gsearch2c.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.82150, std: 0.01148, params: {'min_child_weight': 16},
  mean: 0.82024, std: 0.01097, params: {'min_child_weight': 20},
  mean: 0.81799, std: 0.01404, params: {'min_child_weight': 24},
  mean: 0.81785, std: 0.01523, params: {'min_child_weight': 28},
  mean: 0.81766, std: 0.01268, params: {'min_child_weight': 32},
  mean: 0.81489, std: 0.01345, params: {'min_child_weight': 36},
  mean: 0.81745, std: 0.01247, params: {'min_child_weight': 40}],
 {'min_child_weight': 16},
 0.82150207385972573)

In [30]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=2,
                                        min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch3.fit(data,target)

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=6, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'gamma': [0.0, 0.1, 0.2, 0.3, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [31]:
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.82762, std: 0.00987, params: {'gamma': 0.0},
  mean: 0.82762, std: 0.00987, params: {'gamma': 0.1},
  mean: 0.82762, std: 0.00987, params: {'gamma': 0.2},
  mean: 0.82762, std: 0.00987, params: {'gamma': 0.3},
  mean: 0.82762, std: 0.00987, params: {'gamma': 0.4}],
 {'gamma': 0.0},
 0.82761544081026572)

In [32]:
xgb2 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=7,
        min_child_weight=36,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
# modelfit(xgb2, data)
cross_val(xgb2, data)

TypeError: cross_val() takes 1 positional argument but 2 were given

### Tune subsample and colsample_bytree

In [33]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=2,
                                        min_child_weight=6, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch4.fit(data,target)

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.4, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=6, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [34]:
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.82657, std: 0.00788, params: {'subsample': 0.6, 'colsample_bytree': 0.6},
  mean: 0.82955, std: 0.00549, params: {'subsample': 0.7, 'colsample_bytree': 0.6},
  mean: 0.82783, std: 0.01012, params: {'subsample': 0.8, 'colsample_bytree': 0.6},
  mean: 0.82598, std: 0.00871, params: {'subsample': 0.9, 'colsample_bytree': 0.6},
  mean: 0.82797, std: 0.00865, params: {'subsample': 0.6, 'colsample_bytree': 0.7},
  mean: 0.82902, std: 0.00843, params: {'subsample': 0.7, 'colsample_bytree': 0.7},
  mean: 0.82698, std: 0.00926, params: {'subsample': 0.8, 'colsample_bytree': 0.7},
  mean: 0.82749, std: 0.00561, params: {'subsample': 0.9, 'colsample_bytree': 0.7},
  mean: 0.82800, std: 0.00749, params: {'subsample': 0.6, 'colsample_bytree': 0.8},
  mean: 0.82699, std: 0.00823, params: {'subsample': 0.7, 'colsample_bytree': 0.8},
  mean: 0.82762, std: 0.00987, params: {'subsample': 0.8, 'colsample_bytree': 0.8},
  mean: 0.82519, std: 0.00614, params: {'subsample': 0.9, 'colsample_bytree'

### regularization

In [35]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test6 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=2,
                                        min_child_weight=6, gamma=0.4, subsample=0.7, colsample_bytree=0.6,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch6.fit(data,target)

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.6,
       gamma=0.4, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=6, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.7),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'reg_alpha': [1e-05, 0.01, 0.1, 1, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [36]:
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.82955, std: 0.00549, params: {'reg_alpha': 1e-05},
  mean: 0.82901, std: 0.00474, params: {'reg_alpha': 0.01},
  mean: 0.82887, std: 0.00646, params: {'reg_alpha': 0.1},
  mean: 0.82877, std: 0.00832, params: {'reg_alpha': 1},
  mean: 0.50000, std: 0.00000, params: {'reg_alpha': 100}],
 {'reg_alpha': 1e-05},
 0.82955393083374318)

In [37]:
from sklearn import cross_validation
xgb = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=2,
                    min_child_weight=6, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
                    objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27)
kf = cross_validation.KFold(len(target), n_folds=5)
for train_index, test_index in kf:

   X_train, X_test = data[train_index], data[test_index]
   y_train, y_test = target[train_index], target[test_index]

   xgb.fit(X_train, y_train)
   print(confusion_matrix(y_test, xgb.predict(X_test)))

KeyError: '[ 314  315  316 ..., 1563 1564 1565] not in index'

In [38]:
target['is_spammer'].value_counts()
data.describe()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

## Others

In [39]:
target.value_counts()

AttributeError: 'numpy.ndarray' object has no attribute 'value_counts'

In [40]:
param_test = {
    'gamma':np.linspace(0.05, 1, num=20),
    'kernel':['rbf', 'poly', 'sigmoid']
}
cv = KFold(n_splits=5)
gsearch = GridSearchCV(estimator = SVC(kernel='sigmoid', gamma=1), 
                       param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=cv)
gsearch.fit(data, target)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_

KeyboardInterrupt: 

In [ ]:
# split to train and test sets
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(
    data, target, test_size=0.33, random_state=0)

train_X = train_X.as_matrix()
test_X = test_X.as_matrix()
# train_y = train_y.as_matrix()

pred_scores = []
krnl = {'rbf' : 'rbf','polynominal' : 'poly', 'sigmoid': 'sigmoid'}
for k,v in krnl.items():
    for i in np.linspace(0.05, 1, num=20):
        svc = SVC(kernel=v, gamma=i)
        svc.fit(train_X, train_y)
        pred = svc.predict(test_X)
        pred_scores.append((k, [i, accuracy_score(test_y,pred)]))
df = pd.DataFrame.from_items(pred_scores,orient='index', columns=['Gamma','Score'])
df['Score'].plot(kind='line', figsize=(11,6), ylim=(0.8,1.0))

In [ ]:
df[df['Score'] == df['Score'].max()]